In [32]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow.pyfunc

#### Search models

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [5]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [11]:
runs = client.search_runs(
  experiment_ids='1',
  filter_string="metrics.rmse < 6.5",
  run_view_type=ViewType.ACTIVE_ONLY,
  max_results=5,
  order_by=["metrics.rmse ASC"]
)

In [12]:
for run in runs:
  print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 9371e681e4ec4723b14cac73a9fe82c2, rmse: 6.3053
run id: b36bcd3b81ca46b8a799b776a680ca02, rmse: 6.3062
run id: 96985c72650747f8bef09b75151b2222, rmse: 6.3064
run id: 8e1bf79c52314a1d8830d1ae7311408d, rmse: 6.3069
run id: 9dea084aee9c4779b14cc11ad63f9cc6, rmse: 6.3093


#### Model aliasing

In [14]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [24]:
run_id = "8e1bf79c52314a1d8830d1ae7311408d"
model_name = "nyc-taxi-regressor"
mlflow.register_model(model_uri=f"runs:/{run_id}/model", name=model_name)

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1718658460483, current_stage='None', description=None, last_updated_timestamp=1718658460483, name='nyc-taxi-regressor', run_id='8e1bf79c52314a1d8830d1ae7311408d', run_link=None, source='/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/experiment-tracking/mlruns/1/8e1bf79c52314a1d8830d1ae7311408d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

#### Set model alias

In [26]:
latest_versions = client.search_model_versions(filter_string=f"name='{model_name}'")

In [28]:
for version in latest_versions:
  print(f"version: {version.version}, stage: {version.current_stage}, tags: {version.tags}")

version: 3, stage: None, tags: {}
version: 2, stage: None, tags: {}
version: 1, stage: None, tags: {}


In [29]:
client.set_registered_model_alias(name=model_name, alias="champion", version="1")

#### Loading a registered model

In [33]:
model_version = 1
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:15:02] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [34]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 9dea084aee9c4779b14cc11ad63f9cc6